In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 30.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.1/774.1 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 57.9 MB/s eta 0:00:00a 0:00:01


In [ ]:
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 56.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 39.7 MB/s eta 0:00:0000:0100:01


In [ ]:
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 54.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 40.8 MB/s eta 0:00:00


In [ ]:
!pip install imbalanced-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.0/258.0 kB 10.5 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
from imblearn.over_sampling import RandomOverSampler

In [ ]:
MAX_LEN = 128
BATCH_SIZE = 16
EPOCHS = 3
LEARNING_RATE = 2e-5

tokenizer = BertTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')
model = BertForSequenceClassification.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', num_labels=2)  # 2 classes: Not readmitted, Readmitted


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_df = pd.read_csv("model_data.csv")

model_df = model_df.drop_duplicates(subset=['SUBJECT_ID'], keep='first')

num_records_before_sampling = len(model_df)

In [ ]:
ros = RandomOverSampler(random_state=42)
train_texts_reshaped = model_df['TEXT'].values.reshape(-1, 1)
train_texts_balanced, train_labels_balanced = ros.fit_resample(train_texts_reshaped, model_df['readmitted'])
model_df_balanced = pd.DataFrame({'TEXT': train_texts_balanced.squeeze(), 'readmitted': train_labels_balanced})

num_records_after_sampling = len(model_df_balanced)

print("Number of records before sampling:", num_records_before_sampling)
print("Number of records after sampling:", num_records_after_sampling)

Number of records before sampling: 41965
Number of records after sampling: 81308


In [ ]:
unique_counts = model_df_balanced["readmitted"].value_counts()
print(unique_counts)

readmitted
0    40654
1    40654
Name: count, dtype: int64


In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(model_df_balanced['TEXT'], model_df_balanced['readmitted'], test_size=0.3, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(test_texts, test_labels, test_size=0.5, random_state=42)


class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])
        label = self.labels.iloc[idx]

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_token_type_ids=True
        )

        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'input_ids': torch.tensor(ids, dtype=torch.long),
            'attention_mask': torch.tensor(mask, dtype=torch.long),
            'labels': torch.tensor(label, dtype=torch.long)
        }

train_dataset = CustomDataset(train_texts, train_labels, tokenizer, MAX_LEN)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_dataset = CustomDataset(val_texts, val_labels, tokenizer, MAX_LEN)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

test_dataset = CustomDataset(test_texts, test_labels, tokenizer, MAX_LEN)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(EPOCHS):
    model.train()
    for batch in train_loader:
        try:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

        except Exception as e:
            print(f"Error occurred in training loop: {e}. Skipping this batch.")
            continue

    model.eval()
    val_preds = []
    val_true = []
    with torch.no_grad():
        for batch in val_loader:
            try:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(input_ids, attention_mask=attention_mask)
                _, preds = torch.max(outputs.logits, dim=1)

                val_preds.extend(preds.tolist())
                val_true.extend(labels.tolist())

            except Exception as e:
                print(f"Error occurred in evaluation loop: {e}. Skipping this batch.")
                continue

    val_acc = accuracy_score(val_true, val_preds)
    val_report = classification_report(val_true, val_preds)

    print(f"Epoch {epoch+1}/{EPOCHS}, Validation Accuracy: {val_acc}")
    print(val_report)

Epoch 1/3, Validation Accuracy: 0.9678583142013775
              precision    recall  f1-score   support

           0       1.00      0.94      0.97      6184
           1       0.94      1.00      0.97      6012

    accuracy                           0.97     12196
   macro avg       0.97      0.97      0.97     12196
weighted avg       0.97      0.97      0.97     12196

Epoch 2/3, Validation Accuracy: 0.9812233519186618
              precision    recall  f1-score   support

           0       1.00      0.96      0.98      6184
           1       0.96      1.00      0.98      6012

    accuracy                           0.98     12196
   macro avg       0.98      0.98      0.98     12196
weighted avg       0.98      0.98      0.98     12196

Epoch 3/3, Validation Accuracy: 0.9803214168579862
              precision    recall  f1-score   support

           0       1.00      0.96      0.98      6184
           1       0.96      1.00      0.98      6012

    accuracy                 

In [ ]:
import pickle

filename = 'BERT_model_final.pkl'

with open(filename, 'wb') as file:
    pickle.dump(model, file)

In [ ]:
import pickle

def tokenize_and_convert(data_loader):
    tokenized_data = []
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        tokenized_batch = {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }
        tokenized_data.append(tokenized_batch)
    return tokenized_data

train_tokenized = tokenize_and_convert(train_loader)


In [ ]:
test_tokenized = tokenize_and_convert(test_loader)

In [ ]:
val_tokenized = tokenize_and_convert(val_loader)

In [ ]:
all_tokenized = train_tokenized + test_tokenized + val_tokenized

In [ ]:
with open("tokenizedBERT.pkl", "wb") as f:
    pickle.dump(all_tokenized, f)